In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif,chi2
from sklearn.model_selection import train_test_split

df=pd.read_csv('train.csv')


from datetime import date
import datetime

df['Day'] = df['DATE'].apply(lambda x: 
                      (date(datetime.datetime.strptime(x,"%Y-%m-%d").year,datetime.datetime.strptime(x,"%Y-%m-%d").month,
                      datetime.datetime.strptime(x,"%Y-%m-%d").day)-date(2019,4,10)).days)
df.drop(['DATE'],axis=1,inplace=True)
df

,SITE_ID,CELL_TYPE_Macro,CELL_TYPE_Mobil,CELL_TYPE_TRP,CELL_TYPE_Tx site,CELL_TYPE_micro,N_TRANSPORTED_SITES,GEOGRAPHIC_CLUSTER_K_0,GEOGRAPHIC_CLUSTER_K_1,GEOGRAPHIC_CLUSTER_K_2,...,skew_fire/smoke_alarms_prev14d,skew_ge_alarms_prev14d,skew_power_alarms_prev14d,skew_temperature_alarms_prev14d,kurt_equipment_alarms_prev14d,kurt_fire/smoke_alarms_prev14d,kurt_ge_alarms_prev14d,kurt_power_alarms_prev14d,kurt_temperature_alarms_prev14d,Day
0,146,1,0,0,0,0,3.0,0,0,0,...,0.0,0.0,0.0,0.0,-1.212308,-1.212308,-1.212308,-1.212308,-1.212308,0
1,146,1,0,0,0,0,3.0,0,0,0,...,0.0,0.0,0.0,0.0,-1.212308,-1.212308,-1.212308,-1.212308,-1.212308,1
2,146,1,0,0,0,0,3.0,0,0,0,...,0.0,0.0,0.0,0.0,-1.212308,-1.212308,-1.212308,-1.212308,-1.212308,2
3,146,1,0,0,0,0,3.0,0,0,0,...,0.0,0.0,0.0,0.0,-1.212308,-1.212308,-1.212308,-1.212308,-1.212308,3
4,146,1,0,0,0,0,3.0,0,0,0,...,0.0,0.0,0.0,0.0,-1.212308,-1.212308,-1.212308,-1.212308,-1.212308,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621295,1251,1,0,0,0,0,7.0,0,0,0,...,0.0,0.0,0.0,0.0,-1.212308,-1.212308,-1.212308,-1.210000,-1.212308,295
621296,1251,1,0,0,0,0,7.0,0,0,0,...,0.0,0.0,0.0,0.0,-1.212308,-1.212308,-1.212308,-1.210000,-1.212308,296
621297,1251,1,0,0,0,0,7.0,0,0,0,...,0.0,0.0,0.0,0.0,-1.212308,-1.212308,-1.212308,-1.210000,-1.212308,297
621298,1251,1,0,0,0,0,7.0,0,0,0,...,0.0,0.0,0.0,0.0,-1.212308,-1.212308,-1.212308,-1.210000,-1.212308,298


In [2]:
target_variable='aircon_sum_target_next14d'
input_variables=df.columns[df.columns!=target_variable]


train_data=df.loc[:,input_variables]
test_data=df.loc[:,target_variable]


In [3]:
site_IDs=df['SITE_ID']
Dates=df['Day']
N_Transported_Sites=df[ 'N_TRANSPORTED_SITES']
Cell_Type=df.loc[:, 'CELL_TYPE_Macro': 'CELL_TYPE_micro']
Geographical_Cluster=df.loc[:, 'GEOGRAPHIC_CLUSTER_K_0': 'GEOGRAPHIC_CLUSTER_K_9']
m3_w_prevXd=df.loc[ :, 'mean_temperature_prev7d':  'min_pressure_prev3d']
m3_w_f_nextXd=df.loc[:,'mean_temperature_f_next14d':'min_pressure_f_next7d']
aircon_sum=df.loc[:,'aircon_sum_wo_prev7d':'aircon_sum_target_next14d']
cat_sum_alarms_prevXd=df.loc[:,'equipment_sum_alarms_prev14d':'temperature_sum_alarms_prev3d']
cat_m3_persistance_prevXd=df.loc[:,'equipment_max_persistance_prev7d':'temperature_min_persistance_prev3d']
skew_cat_alarms_prev14d=df.loc[:,'skew_equipment_alarms_prev14d': 'skew_temperature_alarms_prev14d']
kurt_cat_alarms_prev14d=df.loc[:,'kurt_equipment_alarms_prev14d': 'kurt_temperature_alarms_prev14d']
target_data=df.loc[:,'aircon_sum_target_next14d']

In [8]:
from sklearn.preprocessing import StandardScaler    
from sklearn.preprocessing import MinMaxScaler
X=train_data
y=test_data
scaler = MinMaxScaler()
scaler.fit(X)
X=scaler.transform(X)

np.shape(X)

(621300, 135)

In [5]:
# Hold out
X_train,X_test, y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [7]:

# Feature Selection
#X_train = scaler.fit_transform()
selector= SelectKBest(chi2,k=15).fit(X_train,y_train)

# Get idxs of columns to keep
idxs_selected= selector.get_support(indices=True)
idxs_selected
input_variables[idxs_selected]

Index(['CELL_TYPE_micro', 'GEOGRAPHIC_CLUSTER_K_1', 'GEOGRAPHIC_CLUSTER_K_3',
       'GEOGRAPHIC_CLUSTER_K_7', 'GEOGRAPHIC_CLUSTER_K_9',
       'aircon_sum_wo_prev7d', 'aircon_sum_wo_prev14d', 'mean_rain_mm_prev7d',
       'mean_temperature_f_next14d', 'max_temperature_f_next14d',
       'min_temperature_f_next14d', 'mean_temperature_f_next7d',
       'max_temperature_f_next7d', 'min_temperature_f_next7d', 'Day'],
      dtype='object')

In [267]:
#np.shape(X_train[:,idxs_selected])
#X_train[:,idxs_selected]

train_data_selected=train_data.loc[:,input_variables[idxs_selected]]
train_data_selected

,CELL_TYPE_micro,GEOGRAPHIC_CLUSTER_K_1,GEOGRAPHIC_CLUSTER_K_3,GEOGRAPHIC_CLUSTER_K_7,GEOGRAPHIC_CLUSTER_K_9,aircon_sum_wo_prev7d,aircon_sum_wo_prev14d,mean_rain_mm_prev7d,mean_temperature_f_next14d,max_temperature_f_next14d,min_temperature_f_next14d,mean_temperature_f_next7d,max_temperature_f_next7d,min_temperature_f_next7d,Day
0,0,0,0,0,0,0.0,0.0,1.33,8.00,12.0,5.0,6.14,9.0,5.0,0
1,0,0,0,0,0,0.0,0.0,1.90,8.50,16.0,5.0,5.86,8.0,5.0,1
2,0,0,0,0,0,0.0,0.0,4.70,9.36,20.0,5.0,6.29,11.0,5.0,2
3,0,0,0,0,0,0.0,0.0,4.77,10.57,22.0,5.0,6.86,11.0,5.0,3
4,0,0,0,0,0,0.0,0.0,5.39,11.50,22.0,5.0,7.86,12.0,5.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621295,0,0,0,0,1,0.0,0.0,3.57,5.71,11.0,1.0,6.71,11.0,2.0,295
621296,0,0,0,0,1,0.0,0.0,3.89,5.79,11.0,1.0,6.57,11.0,2.0,296
621297,0,0,0,0,1,0.0,0.0,4.04,5.64,11.0,1.0,5.43,11.0,1.0,297
621298,0,0,0,0,1,0.0,0.0,4.04,5.43,9.0,1.0,4.29,9.0,1.0,298


In [10]:
# np.shape(X_train)
X_train_selected_normalized=X_train[:,idxs_selected]

In [12]:
X=X_train_selected_normalized

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y_train, test_size=0.2, random_state=42)

In [17]:
from sklearn import linear_model

regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)


In [18]:
lasso = linear_model.Lasso(0.01, normalize=True)

In [19]:
lasso.fit(X_train, y_train)

Lasso(alpha=0.01, normalize=True)

In [20]:
y_pred_lasso = lasso.predict(X_test)

In [21]:
y_pred_lasso

array([0.00585969, 0.00585969, 0.00585969, ..., 0.00585969, 0.00585969,
       0.00585969])

In [22]:
alphas = [0.00001, 0.0001, 0.001, 0.01, 0.1]


In [23]:
lassocv = linear_model.LassoCV(eps=0.001, n_alphas=100, alphas=alphas, fit_intercept=True, normalize=False, max_iter=10000,  cv=10)


In [24]:
lassocv.fit(X_train, y_train)


LassoCV(alphas=[1e-05, 0.0001, 0.001, 0.01, 0.1], cv=10, max_iter=10000)

In [25]:
y_pred_lassocv = lassocv.predict(X_test)


In [26]:
lassocv.alpha_


1e-05

In [28]:
result = pd.DataFrame(X_test['SITE_ID'])
result['Day']=X_test['Day']
result['NoA']=y_pred_lassocv
result.groupby(['SITE_ID','Day'],as_index=False)['NoA'].sum()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices